In [1]:
%run -i 'classes/libraries.py'

In [2]:
# Import all the necessary libraries used inside the system
from classes import ethData, models 

In [3]:
m = models.models()

In [4]:
df_nasdaq = m.getNasdaqData()

In [5]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from_data = (datetime.now() - relativedelta(years=4)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')

In [6]:
# web is an imported library
df = m.getStockQuote(web, df_nasdaq['symbol'][10], 'yahoo', from_data, today)

In [7]:
df_nasdaq['symbol'][10]

'AAL'

In [8]:
df_filtered = df.filter(['Close'])

In [9]:
df_filtered.tail(math.ceil(df_filtered.shape[0] * 0.2)).max()

Close    25.17
dtype: float64

In [ ]:
models = {}
rmse = {}
for symbol in df_nasdaq['symbol']:
    try:
        df = m.getStockQuote(web, symbol, 'yahoo', from_data, today)
        if(df.notnull and df.shape[0] >= 60):
            print("Ticker Analyzed: {0}".format(symbol))
            print(df.shape)
            df_filtered = df.filter(['Close'])
            perc_train = .8
            models[symbol], rmse[symbol] = m.modelBase(df_filtered, 60, perc_train, 3, False)
            df_test = df_filtered.tail(math.ceil(df_filtered.shape[0] * 1 - perc_train))
            print("Accuracy: {0}".format(rmse[symbol] / (df_test.max() - df_test.min())))
    except:
        print("Error on {0}!".format(symbol))

Ticker Analyzed: A
(1006, 6)
Epoch 1/3
745/745 [==============================] - 17s 20ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0322 - mape: 19.1016
Epoch 2/3
745/745 [==============================] - 15s 20ms/step - loss: 6.5295e-04 - mse: 6.5295e-04 - mae: 0.0193 - mape: 11.4934
Epoch 3/3
745/745 [==============================] - 15s 20ms/step - loss: 5.4705e-04 - mse: 5.4705e-04 - mae: 0.0172 - mape: 10.3479
Accuracy: Close    0.033607
dtype: float64
Ticker Analyzed: AA
(1006, 6)
Epoch 1/3
745/745 [==============================] - 17s 20ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0582 - mape: 32912.3284
Epoch 2/3
745/745 [==============================] - 15s 21ms/step - loss: 0.0020 - mse: 0.0020 - mae: 0.0334 - mape: 3221.9434
Epoch 3/3
745/745 [==============================] - 15s 20ms/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0275 - mape: 36110.2146
Accuracy: Close    0.017969
dtype: float64
Ticker Analyzed: AACG
(1006, 6)
Epoch 1/3
745/745 [========================

In [8]:
df.shape

(1007, 6)

In [13]:
models

{'AAL': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f979ed80fa0>}

In [8]:

data = df.filter(['Close'])
dataset = data.values


training_data_len = math.ceil(len(dataset) * .8) #80/20 or 70/30

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
train_data = scaled_data[0:training_data_len, :]

#Creating training dataset

x_train = []
y_train = []

windowLenght = 60

for i in range(windowLenght, len(train_data)):
    x_train.append(train_data[i - windowLenght:i, 0])
    y_train.append(train_data[i, 0])

# Convert and reshape x_train and y_train to numpy

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

#Build the LSTM Model (50/50/25 are the number of neurons)

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

#Compile the model

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae', 'mape'])

#Train the model

model.fit(x_train, y_train, batch_size = 1, epochs = 1)

#Creating testing dataset

test_data = scaled_data[training_data_len - windowLenght: , :]

#Create dataset x_test and y_test

x_test = []
y_test = dataset[training_data_len: , :]

for i in range(windowLenght, len(test_data)):
    x_test.append(test_data[i - windowLenght:i, 0])

x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

#Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

#Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)



745/745 [==============================] - 17s 20ms/step - loss: 0.0146 - mse: 0.0146 - mae: 0.0701 - mape: 24601.4863


In [10]:
#Plot data
train = data[:training_data_len]
valid = data[training_data_len:]


In [11]:
valid.insert(1,'Predictions',predictions, True)

In [13]:
valid

,Close,Predictions
Date,,
2020-07-13,11.630000,10.723498
2020-07-14,11.570000,10.567863
2020-07-15,13.440000,10.435513
2020-07-16,12.450000,10.474276
2020-07-17,11.910000,10.541644
...,...,...
2021-04-22,20.070000,20.407457
2021-04-23,21.110001,20.050255
2021-04-26,22.040001,19.789940


In [14]:
from sklearn.metrics import accuracy_score

accuracy.score(valid['Close'], valid['Predictions'])


NameError: name 'accuracy' is not defined

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Prediction'], loc = 'lower right')
plt.show()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Close Price HIstory')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize= 18)
plt.ylabel('Current Price USD ($)', fontsize= 18)


In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .8) #80/20

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
#Creating training dataset
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
windowLenght = 60
for i in range(windowLenght, len(train_data)):
    x_train.append(train_data[i - windowLenght:i, 0])
    y_train.append(train_data[i, 0])
    if i <= windowLenght:
        print(x_train)
        print(y_train)
        

In [ ]:
# Convert x_train and y_train to numpy
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the Data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
#Build the LSTM Model (50/50/25 are the number of neurons)

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Train the model
model.fit(x_train, y_train, batch_size = 1, epochs = 100)

In [ ]:
#Creating testing dataset
test_data = scaled_data[training_data_len - windowLenght: , :]
#Create dataset x_test and y_test
x_test = []
y_test = dataset[training_data_len: , :]
for i in range(windowLenght, len(test_data)):
    x_test.append(test_data[i - windowLenght:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
rmse

In [ ]:
#Plot data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.show()